# Feature Engineering

- 신뢰도가 낮은 Feature의 갯수 줄이기

# 데이터 불러오기

In [ ]:
import pandas as pd

df_data = pd.read_csv('trace.csv')
df_data.head()

# 데이터 분석

In [ ]:
df_data.describe()

In [ ]:
# 각 필드별 NaN 갯수 확인
df_data.isna().sum()

In [ ]:
isna_dict = dict(df_data.isna().sum())
isna_keys = list(isna_dict.keys())
del_column_idx = []
del_column_names = []
for idx, cnt in enumerate(isna_dict.values()):
    if cnt >= 90:
        print(idx, cnt, isna_keys[idx])
        del_column_idx.append(idx)
        del_column_names.append(isna_keys[idx])

In [ ]:
print(len(del_column_names), "\n", del_column_names)

In [ ]:
df_data = df_data.drop(del_column_names, axis=1)

In [ ]:
df_data.describe()

# 입력데이터 포멧 맞추기

In [ ]:
import numpy as np
ips_data = np.array(df_data)
ips_data[0]

# AP별로 wifi rssi 신호 특성을 확인

In [ ]:
row = ips_data[0]
print(f"{df_data.columns[1]}/{row[1]}, {df_data.columns[4]}/{row[4]}, {df_data.columns[8]}/{row[8]}")

In [ ]:
import math
wifi_ap_rssis_dict = {}
for row in ips_data:
    for idx, rssi in enumerate(row[4:]):
        mac = df_data.columns[4+idx]
        if mac not in wifi_ap_rssis_dict.keys():
            wifi_ap_rssis_dict[mac] = {'values':[]}
        if math.isnan(rssi) is False:
            wifi_ap_rssis_dict[mac]['values'].append(rssi)

# wifi_ap_rssis_dict = { mac: {"values":[]}}
# k is mac
# v is {'values'}
           
for v in wifi_ap_rssis_dict.values():
    v['avg'] = sum(v['values'])/len(v['values'])
    v['min'] = min(v['values'])
    v['default'] = -120    


In [ ]:
len(wifi_ap_rssis_dict)

In [ ]:
import matplotlib.pyplot as plt
stop_cnt = 5
idx = 1
plt.figure(figsize=(20,4))

for ap, rssis in wifi_ap_rssis_dict.items():
    print(ap, rssis)

    plt.subplot(1,stop_cnt,idx)
    plt.boxplot(rssis['values'])
    
    idx += 1
    if idx > stop_cnt:
        break


# nan 일 경우 해당 AP의 최소 rssi 값으로 대체

In [ ]:
for row in ips_data:
    for idx in range(len(row[4:])):
        col = 4+idx
        mac = df_data.columns[col]
        rssi = row[col]
        if math.isnan(rssi) == True:
            row[col] = wifi_ap_rssis_dict[mac]['min']
                        

In [ ]:
print(ips_data[0])

# Classifier Model 을 위한 데이터 만들기

## STEP1. 입력데이터 포멧 맞추기

In [ ]:
# timestamp, region, latitude, longitude는 training 대상 feature에 서 제외
ips_x = ips_data[:, 4:]
ips_x.shape

In [ ]:
# region을 class 로 활용하기 위해 두번째 열만으로 하나의 배열을 만듬.
ips_y = ips_data[:, 1:2]
ips_y = ips_y.reshape(-1)
ips_y = ips_y.astype(np.int32)
print(ips_y)

## STEP2. Training & Test set 으로 나눠주기

In [ ]:
from sklearn import model_selection

train_x, test_x, train_y, test_y = model_selection.train_test_split(ips_x, ips_y, test_size=0.3, shuffle=False)

print("x_train:", train_x.shape, "y_train:", train_y.shape)
print("x_test: ", test_x.shape, "y_test: ", test_y.shape)

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier() # 모델 생성
model.fit(train_x, train_y)    # 모델 학습

In [ ]:
model.score(test_x, test_y)

In [ ]:
hit = 0
total = 0
results = []
for idx in range(len(test_x)):
    actual = test_y[idx]
    predicted = model.predict(test_x[idx].reshape(1,-1))
    total += 1
    print(f"[{idx:02d}] actual={actual}, predicted={predicted} - {actual==predicted}")
    if actual == predicted[0]:
        results.append(True)
        hit += 1
    else:
        results.append(False)

print(f"Accuracy: {hit/total} ({hit}/{total})")

In [ ]:
colors = ['blue' if r == True else 'red' for r in results]

# 좌표

In [ ]:
latitudes = ips_data[72:, 2:3]
latitudes = latitudes.reshape(-1)
longitudes = ips_data[72:, 3:4]
longitudes = longitudes.reshape(-1)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(latitudes, longitudes, c=colors)
plt.show()

## SVM

In [ ]:
from sklearn import svm

model = svm.SVC() # 모델 생성
model.fit(train_x, train_y)    # 모델 학습

In [ ]:
model.score(test_x, test_y)

In [ ]:
def display(test_x, test_y, models):
    hit = 0
    total = 0
    results = []
    for idx in range(len(test_x)):
        actual = test_y[idx]
        predicted = model.predict(test_x[idx].reshape(1,-1))
        total += 1
        print(f"[{idx:02d}] actual={actual}, predicted={predicted} - {actual==predicted}")
        if actual == predicted[0]:
            results.append(True)
            hit += 1
        else:
            results.append(False)

    print(f"Accuracy: {hit/total} ({hit}/{total})")
    
    return results

In [ ]:
results = display(test_x, test_y, model)
colors = ['blue' if r == True else 'red' for r in results]

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(latitudes, longitudes, c=colors)
plt.show()

## GaussianProcessClassifier

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
model = GaussianProcessClassifier() # 모델 생성
model.fit(train_x, train_y)    # 모델 학습

In [ ]:
model.score(test_x, test_y)

# sklearn classifiers
https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html